In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
%matplotlib widget

In [4]:
from trsm import TRSM, combos

In [5]:
import awkward as ak

In [6]:
filename = f'../../signal/skimmed/NMSSM_XYH_YToHH_6b_MX_700_MY_400_testing_set_skimmed.root'
trsm = TRSM(filename=filename)

-- [INFO] -- /eos/user/s/srosenzw/miniconda3/envs/work/lib/python3.8/runpy.py -- Opening ROOT file ../../signal/skimmed/NMSSM_XYH_YToHH_6b_MX_700_MY_400_testing_set_skimmed.root with columns
-- [INFO] -- /eos/user/s/srosenzw/miniconda3/envs/work/lib/python3.8/runpy.py -- Tree contains 52891 events.
----------------------------------------------------------------------------------------------------
                                            TABLE COLUMNS                                           
----------------------------------------------------------------------------------------------------
jet_pt                            jet_eta                           jet_phi                           
jet_m                             jet_btag                          jet_qgl                           
jet_idx                           jet_hadronFlav                    jet_partonFlav                    
HX_b1_recojet_m                   HX_b1_recojet_pt                  HX_b1_recojet_eta   

In [41]:
combos7 = combos(trsm, 7)

100%|██████████| 52891/52891 [00:33<00:00, 1565.74it/s]


Total events chosen: 8052


In [43]:
combos7.HX_b1_p4[combos7.combo_mask]

<MomentumArray4D [{pt: 97.9, eta: -0.241, ... m: 17.3}] type='8052 * Momentum4D[...'>

In [9]:
HX_b1_mask = ak.to_numpy(ak.flatten(combos7.swapped_out_H == 0))
HX_b2_mask = ak.to_numpy(ak.flatten(combos7.swapped_out_H == 1))
H1_b1_mask = ak.to_numpy(ak.flatten(combos7.swapped_out_H == 2))
H1_b2_mask = ak.to_numpy(ak.flatten(combos7.swapped_out_H == 3))
H2_b1_mask = ak.to_numpy(ak.flatten(combos7.swapped_out_H == 4))
H2_b2_mask = ak.to_numpy(ak.flatten(combos7.swapped_out_H == 5))

In [10]:
from keras.models import model_from_json
from pickle import load

In [11]:
tag = '5hidden_10_15_25_15_10'

In [12]:
json_file = open(f'../models/{tag}/model/model_1.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
loaded_model.load_weights(f'../models/{tag}/model/model_1.h5')
scaler = load(open(f'../models/{tag}/model/scaler_1.pkl', 'rb'))

In [13]:
test_features_7 = scaler.transform(combos7.combo_features)
scores_7 = loaded_model.predict(test_features_7)[:,0]
scores_7.shape

(56364,)

In [14]:
sgnl_scores_7 = scores_7[combos7.sgnl_mask]
bkgd_scores_7 = scores_7[~combos7.sgnl_mask]

In [15]:
import awkward as ak
import numpy as np

In [16]:
combo_mask = combos7.combo_mask
HX_p4 = ak.to_numpy(combos7.HX_p4.m)[combo_mask]
H1_p4 = ak.to_numpy(combos7.H1_p4.m)[combo_mask]
H2_p4 = ak.to_numpy(combos7.H2_p4.m)[combo_mask]

correct_H = np.concatenate((HX_p4, H1_p4, H2_p4))
incorrect_H = ak.to_numpy(ak.flatten(combos7.incorrect_H_p4.m))

correct_H = correct_H.reshape(correct_H.shape[0],)
incorrect_H = incorrect_H.reshape(incorrect_H.shape[0],)

In [17]:
combos7.combo_features[0]

array([ 1.00625000e+02,  9.73125000e+01,  8.66875000e+01,  8.60000000e+01,
        7.61250000e+01,  5.38750000e+01,  4.29382324e-01, -6.96411133e-01,
        9.41406250e-01,  5.15869141e-01, -2.40814209e-01, -1.04980469e+00,
       -2.85498047e+00, -1.36181641e+00,  4.90966797e-01,  2.43701172e+00,
        6.85729980e-02,  2.89599609e+00,  5.78125000e-01,  9.99511719e-01,
        9.49707031e-01,  5.07324219e-01,  5.65917969e-01,  4.86083984e-01,
        1.08426492e+02,  9.76433094e+01,  7.83632129e+01,  9.04464862e+01,
        7.09784337e+01,  5.92654496e+01])

In [18]:
import matplotlib.pyplot as plt
from consistent_plots import hist, hist2d

In [19]:
def norm_hist(arr, bins=100):
    n, b = np.histogram(arr, bins=bins)
    x = (b[:-1] + b[1:]) / 2
    
    return n/n.max(), b, x

In [20]:
fig, ax = plt.subplots()

ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_1, b7, x7 = norm_hist(incorrect_H, bins=np.linspace(0,1000,100))
n7_2, b7, x7 = norm_hist(correct_H, bins=np.linspace(0,1000,100))

hist(ax, x7, weights=n7_1, bins=b7, label='Incorrect')
hist(ax, x7, weights=n7_2, bins=b7, label='Correct')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Dijet Invariant Mass [GeV]')
ax.set_ylabel('AU')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
fig, ax = plt.subplots()

ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_0, b7, x7 = norm_hist(incorrect_H[HX_b1_mask], bins=np.linspace(0,600,100))
n7_1, b7, x7 = norm_hist(incorrect_H[HX_b2_mask], bins=np.linspace(0,600,100))
n7_2, b7, x7 = norm_hist(HX_p4, bins=np.linspace(0,600,100))

hist(ax, x7, weights=n7_0, bins=b7, label='HX b1')
hist(ax, x7, weights=n7_1, bins=b7, label='HX b2')
hist(ax, x7, weights=n7_2, bins=b7, label='True HX')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Dijet Invariant Mass [GeV]')
ax.set_ylabel('AU')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
fig, ax = plt.subplots()

ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_0, b7, x7 = norm_hist(incorrect_H[H1_b1_mask], bins=np.linspace(0,600,100))
n7_1, b7, x7 = norm_hist(incorrect_H[H1_b2_mask], bins=np.linspace(0,600,100))
n7_2, b7, x7 = norm_hist(H1_p4, bins=np.linspace(0,600,100))

hist(ax, x7, weights=n7_0, bins=b7, label='H1 b1')
hist(ax, x7, weights=n7_1, bins=b7, label='H1 b2')
hist(ax, x7, weights=n7_2, bins=b7, label='True H1')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Dijet Invariant Mass [GeV]')
ax.set_ylabel('AU')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
fig, ax = plt.subplots()

ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_0, b7, x7 = norm_hist(incorrect_H[H2_b1_mask], bins=np.linspace(0,600,100))
n7_1, b7, x7 = norm_hist(incorrect_H[H2_b2_mask], bins=np.linspace(0,600,100))
n7_2, b7, x7 = norm_hist(H2_p4, bins=np.linspace(0,600,100))

hist(ax, x7, weights=n7_0, bins=b7, label='H2 b1')
hist(ax, x7, weights=n7_1, bins=b7, label='H2 b2')
hist(ax, x7, weights=n7_2, bins=b7, label='True H2')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Dijet Invariant Mass [GeV]')
ax.set_ylabel('AU')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(10,3))

ax = axs[0]
ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_1, b7, x7 = norm_hist(incorrect_H[scores_7[~combos7.sgnl_mask] <= 0.8], bins=np.linspace(0,1000,100))
n7_2, b7, x7 = norm_hist(incorrect_H[scores_7[~combos7.sgnl_mask] > 0.8], bins=np.linspace(0,1000,100))

hist(ax, x7, weights=n7_1, bins=b7, label='Low score')
hist(ax, x7, weights=n7_2, bins=b7, label='High Score')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel('Incorrect Higgs mass')
ax.set_ylabel('AU')

ax = axs[1]
ax.set_title(r"$^7C_6$ Combinations")

n7_1, b7 = np.histogram(incorrect_H[scores_7[~combos7.sgnl_mask] <= 0.8], bins=np.linspace(0,1000,100))
n7_2, b7 = np.histogram(incorrect_H[scores_7[~combos7.sgnl_mask] > 0.8], bins=np.linspace(0,1000,100))

hist(ax, x7, weights=n7_1, bins=b7, label='Low score')
hist(ax, x7, weights=n7_2, bins=b7, label='High Score')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel('Incorrect Higgs mass')
ax.set_ylabel('AU')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
fig, ax = plt.subplots()

n, x, y, im = hist2d(ax, incorrect_H, scores_7[~combos7.sgnl_mask], xbins=np.linspace(0,500,100))
plt.colorbar(im, ax=ax)

ax.set_ylabel("Assigned Score")

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
fig, ax = plt.subplots()

ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_0, b7, x7 = norm_hist(bkgd_scores_7[HX_b1_mask], bins=np.linspace(0,1.01,100))
n7_1, b7, x7 = norm_hist(bkgd_scores_7[HX_b2_mask], bins=np.linspace(0,1.01,100))

hist(ax, x7, weights=n7_0, bins=b7, label='HX b1')
hist(ax, x7, weights=n7_1, bins=b7, label='HX b2')
# hist(ax, x7, weights=n7_2, bins=b7, label='True H2')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Assigned Score')
ax.set_ylabel('AU')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
fig, ax = plt.subplots()

ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_0, b7, x7 = norm_hist(bkgd_scores_7[H1_b1_mask], bins=np.linspace(0,1.01,100))
n7_1, b7, x7 = norm_hist(bkgd_scores_7[H1_b2_mask], bins=np.linspace(0,1.01,100))

hist(ax, x7, weights=n7_0, bins=b7, label='H1 b1')
hist(ax, x7, weights=n7_1, bins=b7, label='H1 b2')
# hist(ax, x7, weights=n7_2, bins=b7, label='True H2')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Assigned Score')
ax.set_ylabel('AU')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
fig, ax = plt.subplots()

ax.set_title(r"$^7C_6$ Combinations Normalized")

n7_0, b7, x7 = norm_hist(bkgd_scores_7[H2_b1_mask], bins=np.linspace(0,1.01,100))
n7_1, b7, x7 = norm_hist(bkgd_scores_7[H2_b2_mask], bins=np.linspace(0,1.01,100))

hist(ax, x7, weights=n7_0, bins=b7, label='H2 b1')
hist(ax, x7, weights=n7_1, bins=b7, label='H2 b2')
# hist(ax, x7, weights=n7_2, bins=b7, label='True H2')
ax.legend(fontsize='small', loc=9)

ax.set_xlabel(r'Assigned Score')
ax.set_ylabel('AU')

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …